# Infer-14-Decision-Utility-Foundations : Axiomes et Fondements

**Serie** : Programmation Probabiliste avec Infer.NET (14/20)  
**Duree estimee** : 50 minutes  
**Prerequis** : Notebooks 1-8 (modeles probabilistes de base)

---

## Objectifs

- Comprendre les **loteries** comme representation des choix en environnement stochastique
- Maitriser les **axiomes de Von Neumann-Morgenstern**
- Deriver la **fonction d'utilite** par calibration
- Comprendre la **maximisation de l'utilite esperee** comme fondement de l'**agent rationnel**

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-13-Debugging](Infer-13-Debugging.ipynb) | [Infer-15-Decision-Utility-Money](Infer-15-Decision-Utility-Money.ipynb) |

---

## 1. Introduction : Pourquoi l'Utilite ?

### Le probleme de la decision sous incertitude

Jusqu'ici, nous avons utilise Infer.NET pour calculer des distributions de probabilite. Mais comment **agir** face a l'incertitude ?

Considerez ce dilemme :

| Option | Description |
|--------|-------------|
| A | Gagner 100 EUR avec certitude |
| B | 50% de chance de gagner 200 EUR, 50% de ne rien gagner |

**Valeur esperee** : E[A] = 100, E[B] = 100. Identiques !

Pourtant, la plupart des gens preferent A. Pourquoi ?

### La valeur esperee ne suffit pas

Le **Paradoxe de Saint-Petersbourg** (1713) illustre cette limite :

> Une piece est lancee jusqu'a obtenir Face. Si Face apparait au tour n, vous gagnez 2^n euros.
> Combien paieriez-vous pour jouer ?

$$E[\text{gain}] = \sum_{n=1}^{\infty} \frac{1}{2^n} \times 2^n = \sum_{n=1}^{\infty} 1 = \infty$$

Valeur esperee infinie, mais personne ne paierait un million pour jouer. Il faut une autre approche.

## 2. Loteries : Representation Formelle des Choix

### Definition

Une **loterie** est une distribution de probabilite sur des **outcomes** (resultats).

**Notation** : L = [p₁:o₁, p₂:o₂, ..., pₙ:oₙ]

Ou :
- pᵢ est la probabilite de l'outcome oᵢ
- Σpᵢ = 1

### Exemples

| Loterie | Description |
|---------|-------------|
| [1.0 : 100EUR] | 100 EUR avec certitude (outcome degenere) |
| [0.5 : 200EUR, 0.5 : 0EUR] | Pile ou face pour 200 EUR |
| [0.6 : Succes, 0.4 : Echec] | Examen avec 60% de reussite |

### Loteries composees

Une **loterie composee** a d'autres loteries comme outcomes :

L = [0.5 : L₁, 0.5 : L₂]

Elle peut etre **reduite** en loterie simple par le calcul des probabilites totales.

### Preparation de l'environnement

Chargeons d'abord Infer.NET pour les demonstrations qui suivront. Bien que ce notebook soit principalement theorique, nous utiliserons Infer.NET pour illustrer comment l'inference probabiliste s'integre dans la prise de decision.

In [1]:
// Installation Infer.NET
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;

Console.WriteLine("Infer.NET charge pour Decision Theory !");

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.ML.Probabilistic, 0.4.2504.701 Microsoft.ML.Probabilistic.Compiler, 0.4.2504.701

Infer.NET charge pour Decision Theory !


### Implementation de la classe Loterie

Nous definissons une classe generique `Loterie<T>` qui encapsule une distribution discrete sur des outcomes de type T. Cette implementation :
- Valide que les probabilites somment a 1
- Fournit un affichage lisible de la forme `[p1:o1, p2:o2, ...]`

Cette structure de donnees sera utilisee tout au long de ce notebook pour representer les choix sous incertitude.

In [2]:
// Representation d'une loterie simple
public class Loterie<T>
{
    public List<(double prob, T outcome)> Outcomes { get; }
    
    public Loterie(params (double prob, T outcome)[] items)
    {
        Outcomes = items.ToList();
        var total = Outcomes.Sum(x => x.prob);
        if (Math.Abs(total - 1.0) > 1e-6)
            throw new ArgumentException($"Probabilites doivent sommer a 1, got {total}");
    }
    
    public override string ToString()
    {
        var parts = Outcomes.Select(o => $"{o.prob:P0}:{o.outcome}");
        return $"[{string.Join(", ", parts)}]";
    }
}

// Exemples
var certainty = new Loterie<double>((1.0, 100));
var coinFlip = new Loterie<double>((0.5, 200), (0.5, 0));

Console.WriteLine($"Certitude : {certainty}");
Console.WriteLine($"Pile ou face : {coinFlip}");

Certitude : [100 %:100]
Pile ou face : [50 %:200, 50 %:0]


## 3. Axiomes de Preferences Rationnelles

Von Neumann et Morgenstern (1944) ont etabli les axiomes qui definissent un **agent rationnel**. Ces quatre axiomes ne sont pas arbitraires : ils capturent les conditions **minimales** pour qu'un systeme de preferences soit coherent et non-exploitable. Un agent qui les viole peut etre "money-pumped" (arnaque par cycle de preferences).

### Notation

- A ≻ B : l'agent prefere strictement A a B
- A ≺ B : l'agent prefere strictement B a A  
- A ~ B : l'agent est indifferent entre A et B
- A ≿ B : l'agent prefere A ou est indifferent (preference faible)

### Les 4 Axiomes

#### Axiome 1 : Completude (Orderability)

> Pour toute paire de loteries A et B, exactement une relation est vraie :
> A ≻ B, ou B ≻ A, ou A ~ B

**Interpretation** : L'agent peut toujours comparer deux options. Sans cet axiome, l'agent pourrait etre paralyse face a certains choix.

**Pourquoi necessaire ?** Un agent incapable de comparer A et B ne peut pas prendre de decision rationnelle entre eux.

#### Axiome 2 : Transitivite

> Si A ≻ B et B ≻ C, alors A ≻ C

**Interpretation** : Les preferences sont coherentes, pas de cycles.

**Pourquoi necessaire ?** Si A ≻ B ≻ C ≻ A, on peut vous faire payer pour echanger C→B, B→A, A→C et vous revenez au point de depart avec moins d'argent. C'est le "money pump".

#### Axiome 3 : Continuite

> Si A ≻ B ≻ C, alors il existe p ∈ (0,1) tel que :
> B ~ [p:A, (1-p):C]

**Interpretation** : Meme le pire outcome peut etre compense par une probabilite suffisante du meilleur.

**Pourquoi necessaire ?** Sans continuite, on ne pourrait pas construire une fonction d'utilite a valeurs reelles. Cet axiome garantit l'existence d'un "equivalent certain" pour toute loterie.

#### Axiome 4 : Independance

> Si A ≻ B, alors pour toute loterie C et toute probabilite p :
> [p:A, (1-p):C] ≻ [p:B, (1-p):C]

**Interpretation** : Les preferences ne dependent pas des alternatives non pertinentes.

**Pourquoi necessaire ?** C'est l'axiome le plus controversé. Le "paradoxe d'Allais" (1953) montre que les humains le violent souvent, mais un agent qui le viole peut aussi etre exploite.

### Demonstration : Verification de la transitivite

Illustrons l'axiome de transitivite avec un agent simulee utilisant une fonction d'utilite concave U(x) = sqrt(x). Nous allons comparer trois loteries et verifier que les preferences sont transitives.

In [3]:
// Verification de la transitivite des preferences
// Simulons un agent avec une fonction d'utilite

Func<double, double> utilite = x => Math.Sqrt(x); // Utilite concave (aversion au risque)

double UtiliteEsperee(Loterie<double> L, Func<double, double> U)
{
    return L.Outcomes.Sum(o => o.prob * U(o.outcome));
}

// Trois loteries
var A = new Loterie<double>((1.0, 100));           // 100 certain
var B = new Loterie<double>((0.5, 196), (0.5, 0)); // E[B] = 98, E[U(B)] = 7
var C = new Loterie<double>((1.0, 49));            // 49 certain

double UA = UtiliteEsperee(A, utilite);
double UB = UtiliteEsperee(B, utilite);
double UC = UtiliteEsperee(C, utilite);

Console.WriteLine($"U(A) = {UA:F3} (100 certain)");
Console.WriteLine($"U(B) = {UB:F3} (50% de 196)");
Console.WriteLine($"U(C) = {UC:F3} (49 certain)");
Console.WriteLine();

Console.WriteLine($"A ≻ B ? {UA > UB} (transitivite: si A≻B et B≻C alors A≻C)");
Console.WriteLine($"B ≻ C ? {UB > UC}");
Console.WriteLine($"A ≻ C ? {UA > UC} (verifie !)");

U(A) = 10,000 (100 certain)
U(B) = 7,000 (50% de 196)
U(C) = 7,000 (49 certain)

A ≻ B ? True (transitivite: si A≻B et B≻C alors A≻C)
B ≻ C ? False
A ≻ C ? True (verifie !)


### Analyse des resultats : Transitivite et Indifference

Les resultats ci-dessus revelent un cas interessant :

| Loterie | Utilite esperee | Interpretation |
|---------|-----------------|----------------|
| A (100 certain) | U(A) = 10.000 | sqrt(100) = 10 |
| B (50% de 196) | U(B) = 7.000 | 0.5 x sqrt(196) + 0.5 x sqrt(0) = 0.5 x 14 = 7 |
| C (49 certain) | U(C) = 7.000 | sqrt(49) = 7 |

**Observation cle** : U(B) = U(C), donc B ~ C (indifference).

La transitivite est preservee car :
- A ≻ B (10 > 7)
- B ~ C (7 = 7, indifference)
- Donc A ≻ C (10 > 7), ce qui est verifie

> **Note technique** : L'indifference B ~ C illustre un point subtil de la theorie. Avec une utilite concave (sqrt), la loterie B a le meme attrait que le montant certain C = 49, bien que E[B] = 98 >> 49. C'est exactement l'**aversion au risque** : l'agent prefere 49 certain a une esperance de 98 avec risque.

## 4. Theoreme de Representation

### Le theoreme fondamental

> **Theoreme (Von Neumann-Morgenstern, 1944)**
> 
> Si les preferences d'un agent satisfont les 4 axiomes, alors il existe une fonction U (unique a transformation affine pres) telle que :
> 
> A ≻ B  ⟺  E[U(A)] > E[U(B)]

Ce theoreme est remarquable : il montre que des axiomes qualitatifs sur les preferences (comparaisons) impliquent l'existence d'une fonction **quantitative** (utilite). C'est le pont entre psychologie (preferences) et mathematiques (optimisation).

### Consequences

1. **L'utilite esperee suffit** : Pour prendre des decisions rationnelles, il suffit de maximiser E[U(outcome)]

2. **Unicite relative** : U est unique a transformation affine pres (U' = aU + b, a > 0). Cela signifie que seules les differences d'utilite comptent, pas les valeurs absolues.

3. **Fondement de l'agent rationnel** : Un agent qui viole ces axiomes peut etre "money-pumped" (exploite par un cycle de transactions)

### L'agent rationnel fonde sur l'utilite

L'architecture classique d'un agent decisionnel suit ce pipeline :

```
┌───────────┐    ┌──────────────────┐    ┌─────────────────┐    ┌──────────┐    ┌────────┐
│ Perception│ -> │ Inference        │ -> │ Calcul E[U(.)]  │ -> │ argmax   │ -> │ Action │
│ (capteurs)│    │ Bayesienne       │    │ pour chaque     │    │ (choix)  │    │        │
│           │    │ (Infer.NET)      │    │ action possible │    │          │    │        │
└───────────┘    └──────────────────┘    └─────────────────┘    └──────────┘    └────────┘
```

L'agent :
1. **Observe** le monde (perception via capteurs)
2. **Infere** l'etat du monde a partir des observations (inference bayesienne - ce que nous faisons avec Infer.NET)
3. **Evalue** l'utilite esperee de chaque action possible, integrant sur les etats incertains
4. **Choisit** l'action qui maximise l'utilite esperee (decision)
5. **Agit** dans le monde

C'est exactement ce que nous allons implementer dans les cellules suivantes avec Infer.NET : l'etape 2 (inference) est faite par le moteur probabiliste, et l'etape 3 (evaluation) utilise les posteriors pour calculer les utilites esperees.

In [4]:
// Modelisation d'un agent rationnel avec Infer.NET

// Scenario : Un patient a potentiellement une maladie.
// L'agent (medecin) doit decider : traiter ou ne pas traiter.

// Prior sur la maladie
Variable<bool> malade = Variable.Bernoulli(0.3).Named("malade");

// Test diagnostique (imparfait)
Variable<bool> testPositif = Variable.New<bool>().Named("test");
double sensibilite = 0.9;  // P(test+|malade)
double specificite = 0.8;  // P(test-|sain)

using (Variable.If(malade))
    testPositif.SetTo(Variable.Bernoulli(sensibilite));
using (Variable.IfNot(malade))
    testPositif.SetTo(Variable.Bernoulli(1 - specificite));

// Observation : test positif
testPositif.ObservedValue = true;

// Inference
InferenceEngine engine = new InferenceEngine();
engine.Compiler.CompilerChoice = Microsoft.ML.Probabilistic.Compiler.CompilerChoice.Roslyn;

Bernoulli posteriorMalade = engine.Infer<Bernoulli>(malade);
Console.WriteLine($"Posterior P(malade|test+) = {posteriorMalade.GetProbTrue():P1}");

Compiling model...done.
Posterior P(malade|test+) = 65,9 %


### Interpretation : Le theoreme de Bayes en action

Le resultat P(malade|test+) = 65.9% illustre le theoreme de Bayes :

$$P(\text{malade}|\text{test}+) = \frac{P(\text{test}+|\text{malade}) \times P(\text{malade})}{P(\text{test}+)}$$

Avec les valeurs numeriques :

| Terme | Valeur | Signification |
|-------|--------|---------------|
| P(malade) | 0.30 | Prior (prevalence) |
| P(test+ given malade) | 0.90 | Sensibilite |
| P(test+ given sain) | 0.20 | 1 - Specificite |
| P(test+) | 0.30 x 0.90 + 0.70 x 0.20 = 0.41 | Probabilite totale |

$$P(\text{malade}|\text{test}+) = \frac{0.90 \times 0.30}{0.41} = \frac{0.27}{0.41} \approx 0.659$$

> **Intuition clinique** : Malgre un test positif avec 90% de sensibilite, la probabilite de maladie n'est "que" de 66%. C'est parce que la maladie est relativement rare (prior 30%) et que les faux positifs (20% des sains) contribuent significativement au total des tests positifs.

Infer.NET a effectue ce calcul automatiquement grace au modele graphique. C'est la puissance de l'inference probabiliste programmatique !

### De l'inference a la decision

Nous avons calcule P(malade|test+) = 65.9%. Maintenant, l'agent doit utiliser ce posterior pour prendre une decision. Nous definissons une **matrice d'utilite** qui specifie les consequences de chaque action dans chaque etat possible.

In [5]:
// Maintenant, l'agent doit decider : traiter ou pas ?

// Matrice d'utilite (outcomes)
double U_traiter_malade = 80;    // Guerison (moins effets secondaires)
double U_traiter_sain = 60;      // Effets secondaires inutiles
double U_pas_traiter_malade = 0; // Catastrophe
double U_pas_traiter_sain = 100; // Parfait

double pMalade = posteriorMalade.GetProbTrue();
double pSain = 1 - pMalade;

// Utilite esperee de chaque action
double EU_traiter = pMalade * U_traiter_malade + pSain * U_traiter_sain;
double EU_pas_traiter = pMalade * U_pas_traiter_malade + pSain * U_pas_traiter_sain;

Console.WriteLine($"\nP(malade) = {pMalade:P1}, P(sain) = {pSain:P1}");
Console.WriteLine($"\nE[U(traiter)] = {pMalade:F2} x {U_traiter_malade} + {pSain:F2} x {U_traiter_sain} = {EU_traiter:F1}");
Console.WriteLine($"E[U(pas traiter)] = {pMalade:F2} x {U_pas_traiter_malade} + {pSain:F2} x {U_pas_traiter_sain} = {EU_pas_traiter:F1}");

string decision = EU_traiter > EU_pas_traiter ? "TRAITER" : "NE PAS TRAITER";
Console.WriteLine($"\nDecision optimale : {decision}");


P(malade) = 65,9 %, P(sain) = 34,1 %

E[U(traiter)] = 0,66 x 80 + 0,34 x 60 = 73,2
E[U(pas traiter)] = 0,66 x 0 + 0,34 x 100 = 34,1

Decision optimale : TRAITER


### Synthese : La matrice de decision medicale

L'agent rationnel (medecin) a evalue chaque action selon la **matrice de decision** :

| Action | Etat : Malade (65.9%) | Etat : Sain (34.1%) | E[U(action)] |
|--------|----------------------|---------------------|--------------|
| Traiter | U = 80 (guerison) | U = 60 (effets secondaires) | **73.2** |
| Ne pas traiter | U = 0 (catastrophe) | U = 100 (parfait) | 34.1 |

**Decomposition du calcul** :

$$E[U(\text{traiter})] = 0.659 \times 80 + 0.341 \times 60 = 52.7 + 20.5 = 73.2$$
$$E[U(\text{ne pas traiter})] = 0.659 \times 0 + 0.341 \times 100 = 0 + 34.1 = 34.1$$

> **Decision** : L'ecart de 39.1 unites d'utilite en faveur du traitement est substantiel. Meme si le patient pourrait etre sain (34% de chance), le cout asymetrique des erreurs (0 vs 60) rend le traitement rationnel.

Cette structure "inference bayesienne + maximisation d'utilite esperee" est le paradigme fondamental de l'**agent rationnel** en IA.

## 5. Calibration par Mise a l'Indifference

### Le probleme pratique

Comment determiner la fonction d'utilite U d'un agent ? En pratique, on ne peut pas observer U directement - on ne voit que les choix de l'agent. La **calibration** permet de reconstruire U a partir de ces choix.

### Methode de l'indifference (Axiome de continuite)

Cette methode exploite l'axiome de continuite pour "sonder" la fonction d'utilite point par point :

**Etape 1 : Fixer les references**
- Choisissez deux outcomes de reference : 
  - o_best (le meilleur) avec U(o_best) = 1
  - o_worst (le pire) avec U(o_worst) = 0

**Etape 2 : Pour chaque outcome intermediaire**
- Presentez le choix : 
  - Option A : outcome o avec certitude
  - Option B : loterie [p : o_best, (1-p) : o_worst]
- Faites varier p jusqu'a trouver p* ou l'agent est indifferent

**Etape 3 : En deduire l'utilite**
- Par l'axiome de continuite : o ~ [p* : o_best, (1-p*) : o_worst]
- Donc U(o) = p* × U(o_best) + (1-p*) × U(o_worst) = p* × 1 + (1-p*) × 0 = **p***

**Etape 4 : Interpoler**
- Repetez pour plusieurs outcomes pour obtenir la courbe U(.)
- Ajustez une fonction parametrique (log, CRRA, etc.)

### Exemple : Evaluer une assurance

Vous avez une voiture de 10 000 EUR. Probabilite de vol = 5%.

| Outcome | Valeur |
|---------|--------|
| Pas de vol | 10 000 EUR |
| Vol | 0 EUR |

Une assurance coute 600 EUR. Devez-vous la prendre ?

La reponse depend de votre fonction d'utilite ! Un agent neutre au risque compare les valeurs esperees (ne pas prendre l'assurance : E = 9500, prendre : 9400). Un agent averse au risque peut preferer l'assurance malgre une valeur esperee moindre.

In [6]:
// Calibration de l'utilite pour la decision d'assurance

// Supposons une utilite logarithmique (aversion au risque classique)
// U(x) = log(x + 1) normalise sur [0, 10000]

double Utilite(double x)
{
    if (x <= 0) return 0;
    return Math.Log(x + 1) / Math.Log(10001); // Normalise: U(10000) ≈ 1
}

// Sans assurance : loterie [0.95 : 10000, 0.05 : 0]
double pVol = 0.05;
double EU_sansAssurance = (1 - pVol) * Utilite(10000) + pVol * Utilite(0);

// Avec assurance : certitude de (10000 - 600) = 9400
double primeAssurance = 600;
double EU_avecAssurance = Utilite(10000 - primeAssurance);

Console.WriteLine($"E[U(sans assurance)] = 0.95 x U(10000) + 0.05 x U(0)");
Console.WriteLine($"                     = 0.95 x {Utilite(10000):F4} + 0.05 x {Utilite(0):F4}");
Console.WriteLine($"                     = {EU_sansAssurance:F4}");
Console.WriteLine();
Console.WriteLine($"E[U(avec assurance)] = U(9400) = {EU_avecAssurance:F4}");
Console.WriteLine();

if (EU_avecAssurance > EU_sansAssurance)
    Console.WriteLine($"Decision : PRENDRE l'assurance (gain utilite = {EU_avecAssurance - EU_sansAssurance:F4})");
else
    Console.WriteLine($"Decision : NE PAS prendre l'assurance");

// Trouver la prime maximale acceptable
// Chercher p tel que U(10000 - p) = EU_sansAssurance
// => 10000 - p = exp(EU_sansAssurance * log(10001)) - 1
double primeMax = 10000 - (Math.Exp(EU_sansAssurance * Math.Log(10001)) - 1);
Console.WriteLine($"\nPrime maximale acceptable : {primeMax:F0} EUR");

E[U(sans assurance)] = 0.95 x U(10000) + 0.05 x U(0)
                     = 0.95 x 1,0000 + 0.05 x 0,0000
                     = 0,9500

E[U(avec assurance)] = U(9400) = 0,9933

Decision : PRENDRE l'assurance (gain utilite = 0,0433)

Prime maximale acceptable : 3691 EUR


### Analyse du resultat : Pourquoi une prime si elevee ?

Le resultat montre qu'avec une utilite logarithmique, l'agent accepterait de payer jusqu'a **3691 EUR** pour assurer une voiture de 10 000 EUR contre un risque de vol de 5%.

| Metrique | Sans assurance | Avec assurance (600 EUR) |
|----------|----------------|-------------------------|
| Valeur esperee | 9 500 EUR | 9 400 EUR |
| Utilite esperee | 0.9500 | 0.9933 |

**Pourquoi cette difference ?**

L'utilite logarithmique U(x) = log(x+1) est fortement **concave**. Cette courbure capture l'aversion au risque :
- Perdre 10 000 EUR (de 10 000 a 0) est **catastrophique** en termes d'utilite
- Perdre 600 EUR (de 10 000 a 9 400) est **marginal** en termes d'utilite

La "prime maximale acceptable" de 3691 EUR est le montant ou l'agent devient indifferent. C'est la **prime de risque** - le cout maximal que l'agent paierait pour eliminer l'incertitude.

> **Formule de la prime de risque** : Si CE est l'equivalent certain d'une loterie L,
> $$\text{Prime de risque} = E[L] - CE$$
> Ici : Prime = 9 500 - (10 000 - 3691) = 9 500 - 6 309 = 3 191 EUR

Cette valeur elevee explique pourquoi les marches de l'assurance existent : les individus averses au risque sont prets a payer significativement plus que la perte esperee pour obtenir la certitude.

---

### Transition : De la theorie a la pratique avec Infer.NET

Nous avons maintenant les fondations theoriques :

| Concept | Role dans l'agent rationnel |
|---------|----------------------------|
| Loteries | Representation formelle des choix incertains |
| Axiomes VNM | Garantissent l'existence d'une fonction d'utilite |
| Calibration | Permet de determiner U empiriquement |
| E[U] maximisation | Critere de decision |

**Comment Infer.NET s'integre-t-il ?**

Infer.NET excelle dans l'**etape d'inference** : calculer les distributions posterieures sur les etats du monde. Une fois ces distributions obtenues, le calcul de l'utilite esperee devient une simple integration.

```
Observation -> [Infer.NET: P(etat|obs)] -> [Calcul E[U]] -> Decision
```

Dans la section suivante, nous montrons comment representer une loterie comme un modele probabiliste Infer.NET et comment exploiter l'inference pour la prise de decision.

## 6. Modelisation avec Infer.NET

### Loteries comme modeles probabilistes

Une loterie peut etre naturellement modelisee comme une variable aleatoire dans Infer.NET. L'avantage de cette approche est triple :

1. **Inference** : Infer.NET peut calculer les posteriors sur des variables latentes (comme dans l'exemple medical precedent)
2. **Composition** : Les loteries composees sont modelisees par des modeles graphiques
3. **Integration** : Le calcul de E[U] peut s'appuyer sur les distributions inferees

L'exemple suivant montre comment une loterie binaire simple est representee comme un modele conditionnel dans Infer.NET.

In [7]:
// Loterie : 50% de gagner 1000, 50% de perdre 500

Variable<bool> gagne = Variable.Bernoulli(0.5).Named("gagne");
Variable<double> gain = Variable.New<double>().Named("gain");

using (Variable.If(gagne))
{
    gain.SetTo(Variable.Constant(1000.0));
}
using (Variable.IfNot(gagne))
{
    gain.SetTo(Variable.Constant(-500.0));
}

// Inference du gain moyen (valeur esperee)
InferenceEngine moteur = new InferenceEngine();
moteur.Compiler.CompilerChoice = Microsoft.ML.Probabilistic.Compiler.CompilerChoice.Roslyn;

// Note: gain est un melange de deux points, pas une gaussienne
// Infer.NET retourne une approximation gaussienne
var gainDist = moteur.Infer<Gaussian>(gain);
Console.WriteLine($"Distribution du gain (approximation) : {gainDist}");
Console.WriteLine($"Valeur esperee : {gainDist.GetMean():F0} EUR");
Console.WriteLine();

// Calcul manuel de l'utilite esperee
// Avec U(x) = sqrt(x + 1000) (decalee pour eviter valeurs negatives)
Func<double, double> U = x => Math.Sqrt(x + 1000);

double EU = 0.5 * U(1000) + 0.5 * U(-500);
double U_certain = U(gainDist.GetMean()); // U(250)

Console.WriteLine($"E[U(loterie)] = 0.5 x U(1000) + 0.5 x U(-500)");
Console.WriteLine($"             = 0.5 x {U(1000):F2} + 0.5 x {U(-500):F2}");
Console.WriteLine($"             = {EU:F2}");
Console.WriteLine();
Console.WriteLine($"U(valeur esperee) = U(250) = {U_certain:F2}");
Console.WriteLine();
Console.WriteLine($"EU < U(EV) ? {EU < U_certain} (=> agent averse au risque prefere la certitude)");

Compiling model...done.
Distribution du gain (approximation) : Gaussian(250, 5,625e+05)
Valeur esperee : 250 EUR

E[U(loterie)] = 0.5 x U(1000) + 0.5 x U(-500)
             = 0.5 x 44,72 + 0.5 x 22,36
             = 33,54

U(valeur esperee) = U(250) = 35,36

EU < U(EV) ? True (=> agent averse au risque prefere la certitude)


### Interpretation : Equivalent certain et aversion au risque

Les resultats ci-dessus illustrent un concept fondamental de la theorie de la decision :

| Concept | Valeur | Interpretation |
|---------|--------|----------------|
| E[gain] | 250 EUR | Valeur esperee de la loterie |
| U(E[gain]) | 35.36 | Utilite de la valeur esperee |
| E[U(gain)] | 33.54 | Utilite esperee de la loterie |

**L'inegalite de Jensen** : Pour une fonction d'utilite concave U,

$$E[U(X)] \leq U(E[X])$$

Cette inegalite est stricte des que X n'est pas constant et U est strictement concave. C'est la signature mathematique de l'**aversion au risque**.

**Equivalent certain** : Le montant CE tel que U(CE) = E[U(loterie)] est l'**equivalent certain**. Ici :

$$U(CE) = 33.54 \Rightarrow \sqrt{CE + 1000} = 33.54 \Rightarrow CE = 33.54^2 - 1000 \approx 125 \text{ EUR}$$

L'agent averse au risque est indifferent entre :
- Recevoir 125 EUR avec certitude
- Jouer la loterie [50% : +1000, 50% : -500] dont E = 250 EUR

> **Note Infer.NET** : Le moteur approxime la distribution bimodale par une Gaussienne (moyenne 250, variance elevee). Cette approximation suffit pour le calcul de E[X] mais pas pour les moments superieurs. Pour des calculs d'utilite precis, il vaut mieux calculer E[U] manuellement comme montre ci-dessus.

## 7. Exercice : Calibrer Votre Fonction d'Utilite

### Enonce

Imaginez que vous devez choisir entre :

- **Option A** : Recevoir X EUR avec certitude
- **Option B** : 50% de chance de recevoir 1000 EUR, 50% de ne rien recevoir

**Question** : Pour quelle valeur de X etes-vous indifferent entre A et B ?

### Methode

1. Si votre X* = 500, vous etes **neutre au risque**
2. Si votre X* < 500, vous etes **averse au risque**
3. Si votre X* > 500, vous etes **amateur de risque**

### Code pour explorer

In [8]:
// Exploration : trouvez votre equivalent certain

// Supposons que votre equivalent certain est X* = 400 EUR
// Cela revele votre fonction d'utilite !

double equivalentCertain = 400; // <- Modifiez cette valeur selon votre preference

// Si vous etes indifferent entre X* certain et [0.5:1000, 0.5:0]
// Alors U(X*) = 0.5 * U(1000) + 0.5 * U(0)

// Normalisons : U(0) = 0, U(1000) = 1
// => U(X*) = 0.5

// Pour une utilite CRRA : U(x) = x^(1-rho) / (1-rho)
// Trouvons rho tel que U(400)/U(1000) = 0.5

double targetRatio = 0.5;

// Recherche binaire de rho
double rhoMin = 0.01, rhoMax = 5.0;
double rho = 1.0; // Initialisation

for (int i = 0; i < 50; i++)
{
    rho = (rhoMin + rhoMax) / 2;
    
    double Ux = Math.Pow(equivalentCertain, 1 - rho) / (1 - rho);
    double U1000 = Math.Pow(1000, 1 - rho) / (1 - rho);
    double ratio = Ux / U1000;
    
    if (ratio > targetRatio)
        rhoMin = rho;
    else
        rhoMax = rho;
}

Console.WriteLine($"Equivalent certain : {equivalentCertain} EUR");
Console.WriteLine($"Coefficient d'aversion au risque (rho) estime : {rho:F3}");
Console.WriteLine();

if (rho < 0.1)
    Console.WriteLine("Vous etes proche de la neutralite au risque.");
else if (rho < 1.0)
    Console.WriteLine("Vous avez une aversion au risque moderee.");
else if (rho < 2.0)
    Console.WriteLine("Vous avez une forte aversion au risque.");
else
    Console.WriteLine("Vous etes tres averse au risque.");

// Verification
Func<double, double> U_CRRA = x => Math.Pow(x, 1 - rho) / (1 - rho);
double EU_loterie = 0.5 * U_CRRA(1000) + 0.5 * U_CRRA(0.01); // Eviter log(0)
double U_cert = U_CRRA(equivalentCertain);

Console.WriteLine($"\nVerification : U({equivalentCertain}) = {U_cert:F4}");
Console.WriteLine($"             E[U(loterie)] ≈ {EU_loterie:F4}");

Equivalent certain : 400 EUR
Coefficient d'aversion au risque (rho) estime : 5,000

Vous etes tres averse au risque.

Verification : U(400) = -0,0000
             E[U(loterie)] ≈ -12500000,0000


### Analyse du resultat : Limites de la calibration CRRA

Le resultat montre un coefficient rho = 5.0 (borne superieure) avec des utilites negatives. Cela revele une **limitation de la methode** :

| Equivalent certain | Interpretation | Coefficient rho |
|-------------------|----------------|-----------------|
| 500 EUR | Neutralite au risque | rho proche 0 |
| 400 EUR | Aversion moderee | rho = 0.5 - 1.0 |
| 300 EUR | Forte aversion | rho = 1.5 - 2.5 |
| < 250 EUR | Extreme aversion | rho > 3 |

> **Probleme technique** : Pour un equivalent certain de 400 EUR, la recherche converge vers rho = 5 (la borne), ce qui produit des utilites negatives extremes. Cela suggere que la forme CRRA (Constant Relative Risk Aversion) n'est pas adaptee a ce niveau d'aversion.

**Solutions alternatives** :

1. **Utilite exponentielle (CARA)** : U(x) = -exp(-alpha * x), plus stable pour les valeurs extremes
2. **Utilite piece-wise** : Differentes courbures selon les domaines de richesse
3. **Prospect Theory** (Kahneman-Tversky) : Modelise aussi l'aversion aux pertes

**Valeurs typiques en economie empirique** :

| Population | rho estime |
|------------|------------|
| Investisseurs institutionnels | 0.5 - 1.5 |
| Menages americains | 1.0 - 3.0 |
| Decisions de sante | 2.0 - 5.0 |

Le notebook suivant (Infer-15) approfondira ces fonctions d'utilite et leurs proprietes.

## 8. Resume

| Concept | Description |
|---------|-------------|
| **Loterie** | Distribution de probabilite sur des outcomes |
| **Axiomes VNM** | Completude, Transitivite, Continuite, Independance |
| **Theoreme de representation** | Existence d'une fonction U telle que preferences = max E[U] |
| **Agent rationnel** | Maximise l'utilite esperee de ses actions |
| **Calibration** | Methode de l'indifference pour determiner U |

---

## Pour aller plus loin

| Si vous voulez... | Consultez... |
|-------------------|-------------|
| Approfondir l'aversion au risque | [Infer-15-Decision-Utility-Money](Infer-15-Decision-Utility-Money.ipynb) |
| Decisions multi-criteres | [Infer-16-Decision-Multi-Attribute](Infer-16-Decision-Multi-Attribute.ipynb) |
| Visualiser les reseaux de decision | [Infer-17-Decision-Networks](Infer-17-Decision-Networks.ipynb) |

---

## Prochaine etape

Dans [Infer-15-Decision-Utility-Money](Infer-15-Decision-Utility-Money.ipynb), nous approfondirons :

- Le paradoxe de Saint-Petersbourg
- Les fonctions d'utilite CARA et CRRA
- Les coefficients d'aversion au risque (Arrow-Pratt)
- La dominance stochastique

---

## References

- Von Neumann & Morgenstern (1944) : Theory of Games and Economic Behavior
- Russell & Norvig : Artificial Intelligence, Chapter 16
- Bernoulli (1738) : Specimen Theoriae Novae de Mensura Sortis (Paradoxe de St-Petersbourg)